In [1]:
import getpass
import os
os.environ["COHERE_API_KEY"] = getpass.getpass()

In [2]:
import json

with open("gods.json", "r", encoding="utf-8") as file:
    gods = json.load(file)
print('Total number of items:', len(gods))
print('First item:', gods[0])

Total number of items: 14
First item: {'name': 'زئوس', 'description': 'زئوس بر کوه الیمپوس حکومت می\u200cکند و پادشاه خدایان است. او پسر کرونوس و ریا، شوهر هرا است، و 15 کودک دارد که همه آنها المپیایی هستند.', 'appearance': 'او به عنوان یک مرد قد بلند و ستبر با لباس فاخر نشان داده شده است. او دارای موی بلند و ریش بلند است. به طور کلی، او به صورت ایستاده به تصویر درآمده است.', 'features': 'رعد، اسلحه او و ویژگی اصلی وی است که به طور کلی در دست راست او قرار دارد. گاهی اوقات او یک عصای سلطنتی را در دست چپ خود حمل می\u200cکند. گاو و عقاب حیوانات مقدس او هستند.'}


In [3]:
gods_text = ""
for god in gods:
    god_description = f"{god['name']}: {god['description']} {god['appearance']} {god['features']}"
    gods_text += god_description + "\n\n"

system_message = f"اطلاعات خدایان:\n\n{gods_text}\nپاسخ باید در حداکثر 4 الی 5 کلمه خلاصه شود!\nاگر نتوانستی در 4 الی 5 کلمه پاسخ بدی با رشته ی «نمی‌دانم» پاسخ بده.\nاگر پاسخ سوالی را نمیدانی با رشته ی «نمی‌دانم» پاسخ بده."

In [4]:
examples = [
    {"question": "خدای جنگ کیست؟", "answer": "آرس"},
    {"question": "آتنا معمولاً چه چیزهایی را در دستانش دارد؟", "answer": "سپر و شمشیر"},
    {"question": "آرتمیس دوقلوی چه کسی است؟", "answer": "آپولون"},
    {"question": "زئوس چند کودک دارد که المپیایی هستند؟", "answer": "15"},
    {"question": "چه کسی پادشاه خدایان است؟", "answer": "زئوس"},
    {"question": "سلاح اصلی زئوس چیست؟", "answer": "رعد"},
    {"question": "چه حیواناتی مقدس زئوس هستند؟", "answer": "گاو و عقاب"},
    {"question": "چه کسی خدای مرگ است؟", "answer": "هادس"},
    {"question": "حیوان همراه هادس چیست؟", "answer": "کربروس"},
    {"question": "چه کسی ارباب دریا است؟", "answer": "پوسوئیدون"},
    {"question": "سلاح پوسوئیدون چیست؟", "answer": "نیزه سه‌شاخ"},
    {"question": "هرا الهه چه چیزی است؟", "answer": "ازدواج"},
    {"question": "حیوان مقدس هرا چیست؟", "answer": "طاووس"},
    {"question": "دمتر الهه چه چیزی است؟", "answer": "کشاورزی"},
    {"question": "هستیا الهه چه چیزی است؟", "answer": "خانه"},
    {"question": "آرس خدای چه چیزی است؟", "answer": "جنگ"},
    {"question": "سلاح‌های آرس چه هستند؟", "answer": "سپر و شمشیر"},
    {"question": "هرمس خدای چه چیزهایی است؟", "answer": "سفر و تجارت"},
    {"question": "حیوانات مقدس هرمس کدامند؟", "answer": "لاک‌پشت و خرگوش"},
    {"question": "دیونیسوس خدای چه چیزی است؟", "answer": "شراب و جشن"},
    {"question": "نماد دیونیسوس چیست؟", "answer": "فنجان"},
    {"question": "هفایستوس خدای چه چیزی است؟", "answer": "آهنگری"},
    {"question": "سلاح یا ابزار هفایستوس چیست؟", "answer": "چکش"},
    {"question": "آپولون خدای چه چیزهایی است؟", "answer": "موسیقی و پزشکی"},
    {"question": "آرتمیس الهه چه چیزی است؟", "answer": "شکار"},
    {"question": "سلاح آرتمیس چیست؟", "answer": "تیر و کمان"},
    {"question": "آفرودیت الهه چه چیزی است؟", "answer": "عشق"},
    {"question": "پرنده مقدس آفرودیت چیست؟", "answer": "کبوتر"},
    {"question": "آتنا الهه چه چیزی است؟", "answer": "حکمت و جنگ"},
    {"question": "حیوان همراه آتنا چیست؟", "answer": "جغد"}
]

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(model='embed-multilingual-v3.0')
to_vectorize = [" ".join(example.values()) for example in examples]
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [6]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=5,
)

example_selector.select_examples({"question": "در مورد زئوس بگو"})

[{'answer': 'زئوس', 'question': 'چه کسی پادشاه خدایان است؟'},
 {'answer': 'گاو و عقاب', 'question': 'چه حیواناتی مقدس زئوس هستند؟'},
 {'answer': '15', 'question': 'زئوس چند کودک دارد که المپیایی هستند؟'},
 {'answer': 'رعد', 'question': 'سلاح اصلی زئوس چیست؟'},
 {'answer': 'شراب و جشن', 'question': 'دیونیسوس خدای چه چیزی است؟'}]

In [7]:
from langchain_core.prompts import ChatPromptTemplate

few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["question"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{question}"), ("ai", "{answer}")]
    )
)

prompt = ChatPromptTemplate.from_messages(
    [
     ("system", system_message),
     few_shot_prompt,
     ("human", "{question}")
    ]
)

In [8]:
from langchain_cohere import ChatCohere

model = ChatCohere(top_p=0.9, tempreature=0)

In [10]:
from langchain_core.output_parsers import StrOutputParser
chain = prompt | model | StrOutputParser()

In [ ]:
import pandas as pd

questions = pd.read_csv("questions.csv", encoding="utf-8")

<generator object DataFrame.iterrows at 0x00000199BF56B230>


In [27]:
import time

# لیستی برای ذخیره پاسخ‌ها
answers = []

# تعداد درخواست‌ها
request_count = 0
max_requests = 10  # تعداد درخواست‌هایی که قبل از محدودیت می‌توان ارسال کرد

# پردازش سوالات و تولید پاسخ‌ها
for index, row in questions.iterrows():
    question = row["question"]  # استخراج سوال از دیتافریم
    
    try:
        answer = chain.invoke(question)  # تولید پاسخ با chain.invoke
        answers.append({"answer": answer})  # ذخیره فقط پاسخ‌ها
    except Exception as e:
        print(f"Error occurred: {e}")
        answers.append({"answer": "Error occurred"})  # در صورت خطا یک پیام جایگزین ذخیره می‌کنیم

    # شمارش تعداد درخواست‌ها
    request_count += 1

    # اگر تعداد درخواست‌ها به 10 رسید، یک دقیقه توقف می‌کنیم
    if request_count >= max_requests:
        print("Reached API request limit. Sleeping for 1 minute...")
        time.sleep(60)  # توقف برای یک دقیقه
        request_count = 0  # بازنشانی شمارنده درخواست‌ها

# تبدیل لیست پاسخ‌ها به دیتافریم با فقط یک ستون "answer"
submission = pd.DataFrame(answers)

# نمایش نتیجه
print(submission.head())  # نمایش چند ردیف اول دیتافریم برای بررسی


Reached API request limit. Sleeping for 1 minute...
Reached API request limit. Sleeping for 1 minute...
Reached API request limit. Sleeping for 1 minute...
Reached API request limit. Sleeping for 1 minute...
Reached API request limit. Sleeping for 1 minute...
Reached API request limit. Sleeping for 1 minute...
Reached API request limit. Sleeping for 1 minute...
Reached API request limit. Sleeping for 1 minute...
                      answer
0                        هرا
1                        جغد
2  سفر، تجارت، زبان، حیوانات
3                  مار و گاو
4            لاک‌پشت و خرگوش


In [28]:
print(submission)

                                               answer
0                                                 هرا
1                                                 جغد
2                           سفر، تجارت، زبان، حیوانات
3                                           مار و گاو
4                                     لاک‌پشت و خرگوش
..                                                ...
84                                           نمی‌دانم
85                                                اسب
86                                    آهنگری و فناوری
87  خدای موسیقی، پزشکی، سلامتی، نور، حقیقت و تیران...
88                               لذت، شادی، شراب، جشن

[89 rows x 1 columns]
